# LowLou metric

In [ ]:
#| default_exp lowlow_metric

$$
C_\text{vec} = \frac{\sum_i \mathbf{B}_i \cdot \mathbf{b}_i}{(\sum_i |\mathbf{B}_i|^2 \sum_i |\mathbf{b}_i|^2)^{1/2}}
$$

$$
C_\text{CS} = \frac{1}{M} \sum_i \frac{\mathbf{B}_i \cdot \mathbf{b}_i}{|\mathbf{B}_i||\mathbf{b}_i|}
$$

$$
E'_n = 1 - \frac{\sum_i |\mathbf{B}_i - \mathbf{b}_i|}{\sum_i |\mathbf{b}_i|}
$$

$$
E'_m = 1 - \frac{1}{M} \frac{\sum_i |\mathbf{B}_i - \mathbf{b}_i|}{\sum_i |\mathbf{b}_i|}
$$

$$
\epsilon = \frac{\sum_i |\mathbf{B}_i|^2}{\sum_i |\mathbf{b}_i|^2}
$$

$$
\epsilon_p = \frac{\sum_i |\mathbf{B}_i|^2}{\sum_i |\mathbf{B}_{\text{p}, i}|^2}
$$

$$
\text{CW}_\text{sin} = \frac{\sum_i \frac{|\mathbf{J}_i \times \mathbf{B}_i|}{|\mathbf{B}_i|}}{\sum_i |\mathbf{J}_i|}
$$

In [ ]:
#| export
import numpy as np 
import pandas as pd

In [ ]:
#| export
def vector_norm(vector):
    return np.sqrt((vector ** 2).sum(-1))
    
def divergence(b_field):  # (x, y, z, (xyz-field))
    div_B = np.stack([np.gradient(b_field[..., i], axis=i, edge_order=2) for i in range(3)], axis=-1).sum(-1)
    return div_B

def curl(b_field):  # (x, y, z)
    _, dFx_dy, dFx_dz = np.gradient(b_field[..., 0], axis=[0, 1, 2], edge_order=2)
    dFy_dx, _, dFy_dz = np.gradient(b_field[..., 1], axis=[0, 1, 2], edge_order=2)
    dFz_dx, dFz_dy, _ = np.gradient(b_field[..., 2], axis=[0, 1, 2], edge_order=2)

    rot_x = dFz_dy - dFy_dz
    rot_y = dFx_dz - dFz_dx
    rot_z = dFy_dx - dFx_dy

    return np.stack([rot_x, rot_y, rot_z], -1)

In [ ]:
#| export
def metrics(B, b, B_potential):
    """
    B is the numerical solution
    b is the reference magnetic field
    """

    c_vec = np.sum((B * b).sum(-1)) / np.sqrt((B ** 2).sum(-1).sum() * (b ** 2).sum(-1).sum())

    M = np.prod(B.shape[:-1])
    c_cs = (1 / M) * np.sum((B * b).sum(-1) / (vector_norm(B)*vector_norm(b)))

    E_n = vector_norm(B - b).sum() / vector_norm(b).sum()

    E_m = (1 / M) * (vector_norm(B - b) / vector_norm(b)).sum()

    eps = (vector_norm(B) ** 2).sum() / (vector_norm(b) ** 2).sum()

    eps_p = (vector_norm(B) ** 2).sum() / (vector_norm(B_potential) ** 2).sum()

    j = curl(B)
    sig_J = (vector_norm(np.cross(j, B, -1)) / vector_norm(B)).sum() / vector_norm(j).sum()
    L1 = (vector_norm(np.cross(j, B, -1)) ** 2 / vector_norm(B) ** 2).mean()
    L2 = (divergence(B) ** 2).mean()
    curlB = vector_norm(j).sum() / vector_norm(curl(b)).sum()

    key = ["C_vec", "C_cs", "1-En", "1-Em", "eps", "eps_p", "sig_J", "L1", "L2", "curlB"]
    metric = [c_vec, c_cs, 1-E_n, 1-E_m, eps, eps_p, sig_J, L1, L2, curlB]
    return dict(zip(key, metric))

In [ ]:
#| export
def metric_df(B, b, B_potential, iteration):
    metric = metrics(B=B, b=b, B_potential=B_potential)
    iterinfo = {'iteration': iteration}
    metric = {**iterinfo, **metric}

    df = pd.DataFrame.from_dict([metric])
    return df

In [ ]:
import pyvista as pv

In [ ]:
b_vtk_path = '/userhome/jeon_mg/workspace/_data/lowlou_64_64_64/b.vtk'
b_mesh = pv.read(b_vtk_path)
Nx, Ny, Nz = b_mesh.dimensions
b = b_mesh['B'].reshape(Nz, Ny, Nx, 3).transpose(2, 1, 0, 3)

In [ ]:
b_pot_vtk_path = '/userhome/jeon_mg/workspace/_data/lowlou_64_64_64/b_potential.vtk'
b_pot_mesh = pv.read(b_pot_vtk_path)
b_pot = b_pot_mesh['B'].reshape(Nz, Ny, Nx, 3).transpose(2, 1, 0, 3)

In [ ]:
df_b = metric_df(B=b, b=b, B_potential=b_pot, iteration='Low Lou')
df_bp = metric_df(B=b_pot, b=b, B_potential=b_pot, iteration='Potential')
df = pd.concat([df_b, df_bp], ignore_index=True)
df

,iteration,C_vec,C_cs,1-En,1-Em,eps,eps_p,sig_J,L1,L2,curlB
0,Low Lou,1.000000,1.00000,1.000000,1.000000,1.000000,1.61897,0.167531,9.713042e-02,0.228859,1.000000e+00
1,Potential,0.869229,0.83444,0.437791,0.344632,0.617677,1.00000,0.835331,6.041180e-13,0.105334,3.070785e-07


In [ ]:
df_pinn = df.copy()
df_spinn = df.copy()

In [ ]:
import os

In [ ]:
iterations = np.arange(0, 10001, 1000)

for epo in iterations:
    output_path = '/userhome/jeon_mg/workspace/_data/lowlou_64_64_64'

    B_pinn_vtk_path = os.path.join(output_path, f'PINN/vtk/B_pinn_{epo}.vtk')
    B_pinn_mesh = pv.read(B_pinn_vtk_path)
    B_pinn = B_pinn_mesh['B'].reshape(Nz, Ny, Nx, 3).transpose(2, 1, 0, 3)

    df_new_pinn = metric_df(B=B_pinn, b=b, B_potential=b_pot, iteration=epo)
    df_pinn = pd.concat([df_pinn, df_new_pinn], ignore_index=True)

    B_spinn_vtk_path = os.path.join(output_path, f'SPINN/vtk/B_spinn_{epo}.vtk')
    B_spinn_mesh = pv.read(B_spinn_vtk_path)
    B_spinn = B_spinn_mesh['B'].reshape(Nz, Ny, Nx, 3).transpose(2, 1, 0, 3)

    df_new_spinn = metric_df(B=B_spinn, b=b, B_potential=b_pot, iteration=epo)
    df_spinn = pd.concat([df_spinn, df_new_spinn], ignore_index=True)

In [ ]:
df_pinn

,iteration,C_vec,C_cs,1-En,1-Em,eps,eps_p,sig_J,L1,L2,curlB
0,Low Lou,1.000000,1.000000,1.000000,1.000000,1.000000,1.618970,0.167531,9.713042e-02,0.228859,1.000000e+00
1,Potential,0.869229,0.834440,0.437791,0.344632,0.617677,1.000000,0.835331,6.041180e-13,0.105334,3.070785e-07
2,0,-0.022281,-0.069568,-1.145135,-4.968318,0.449114,0.727102,0.549810,1.361726e-05,0.000030,1.749258e-02
3,1000,0.782732,0.064554,-0.532228,-2.755214,1.982190,3.209107,0.700165,3.529831e+00,3.818808,3.895625e+00
4,2000,0.861569,0.269729,-0.153347,-2.831840,1.077445,1.744351,0.688776,1.160900e+00,0.534191,2.387781e+00
5,3000,0.958774,0.205896,0.415525,-0.689128,0.894947,1.448892,0.576209,5.022586e-01,0.256559,1.625998e+00
6,4000,0.978433,0.828739,0.591337,-0.101339,0.973084,1.575395,0.477454,3.145949e-01,0.161303,1.430187e+00
7,5000,0.985496,0.706799,0.681611,0.340998,0.913434,1.478822,0.413662,2.142373e-01,0.098349,1.232741e+00
8,6000,0.980254,0.167050,0.569098,-0.109252,0.875357,1.417177,0.389427,1.591203e-01,0.066491,1.147264e+00
9,7000,0.982380,0.330822,0.592788,-0.117078,0.907482,1.469187,0.357373,1.295624e-01,0.056362,1.122622e+00


In [ ]:
df_spinn

,iteration,C_vec,C_cs,1-En,1-Em,eps,eps_p,sig_J,L1,L2,curlB
0,Low Lou,1.000000,1.000000,1.000000,1.000000,1.000000,1.618970,0.167531,9.713042e-02,0.228859,1.000000e+00
1,Potential,0.869229,0.834440,0.437791,0.344632,0.617677,1.000000,0.835331,6.041180e-13,0.105334,3.070785e-07
2,0,-0.004380,0.258906,-0.301467,-1.719066,0.090820,0.147035,0.999542,2.133520e-02,0.005343,3.315969e-01
3,1000,0.988603,0.920137,0.727949,0.517681,0.830896,1.345196,0.141347,8.709306e-03,0.004463,8.421051e-01
4,2000,0.991219,0.912797,0.748474,0.528341,0.845375,1.368636,0.107239,6.785732e-03,0.003728,8.751937e-01
5,3000,0.991627,0.929770,0.752738,0.538291,0.854328,1.383131,0.075999,5.406107e-03,0.002917,8.452431e-01
6,4000,0.991515,0.931770,0.750840,0.535096,0.856346,1.386398,0.068946,5.168721e-03,0.003192,8.404825e-01
7,5000,0.991537,0.929238,0.749878,0.529367,0.858588,1.390029,0.065333,5.469157e-03,0.003376,8.398859e-01
8,6000,0.991704,0.931490,0.746774,0.527952,0.835433,1.352541,0.069527,5.779790e-03,0.003731,8.333858e-01
9,7000,0.990934,0.928763,0.755228,0.534240,0.889764,1.440501,0.073513,8.982357e-03,0.008650,8.640849e-01


In [ ]:
df_pinn.to_csv(os.path.join(output_path, 'metric_pinn.csv'), index=False)
df_spinn.to_csv(os.path.join(output_path, 'metric_spinn.csv'), index=False)